# Text segmentation using Hidden Markov Models

In [60]:
import os
import numpy as np
from scipy.special import logsumexp

## Task : automatic segmentation of mails, problem statement 

### Q1 : Give the value of the π vector of the initial probabilities

The model chosen is an HMM (A, B, π) with two states, one (state 1) for
the header, the other (state 2) for the body.

As it is said in the text, the HMM is necessarily in state 1 at the beginning we have that the initial probability vector $\pi$ is : 
$$ \pi = (1, 0)$$


In [61]:
pi = np.array([1,0])

### Q2 : What is the probability to move from state 1 to state 2 ? What is the probability to remain in state 2 ? What is the lower/higher probability ? Try to explain why.

In [62]:
A = np.array([[0.999218078035812,0.000781921964187974],[0,1]])

We have $A_{i,j} = P(j|i)$, so the probability of transition are in the matrix A - called transition matrix.
The probability to move from state 1 to state 2 is $P(2|1) = A_{1,2} = A[0,1] = 0.000781921964187974 $
The probability to remain in state 2 is $P(2|2) = A_{2,2} = A[1,1] = 1 $
$P(2|2)$ is higher and it is normal because if we find a character from the body the next part is necessarily in the body. 

### Q3 : What is the size of B ?

B is the observation probability matrix of size number of symbol times number of state, so here it is : $N \times 2 = 256 \times 2$ for 256 rows and 2 columns.

### Q4 : print the track and present and discuss the results obtained on mail11.txt to mail30.txt

In [63]:
def Viterbi(Obs,A,B,pi):
    N = A.shape[1] # number of states
    T = Obs.shape[0] # number of observations

    states = np.arange(0,N)
    V = np.zeros((N,T))
    back_pointeur = np.zeros((N,T))
    eps = np.finfo(np.double).tiny

    A_prime = np.log(A + eps)
    B_prime = np.log(B + eps)
    pi_prime = np.log(pi + eps)
    
    V[:,0] = pi_prime[:] + B_prime[Obs[0],:]
    
    for t in range(1, T):
        for s in range(N): 
            
            L = [V[i,t-1]+A_prime[i,s] for i in range(N)]
            V[s,t] = max(L) + B_prime[Obs[t],s]
            back_pointeur[s,t-1] = L.index(max(L)) + 1 
    
    L = [V[i,-1] for i in range(N)]
    best_pathprob = max(L)
    best_pathpointer = L.index(best_pathprob) + 1
    path = np.zeros(T, dtype=int)   
    path[T-1] = best_pathpointer
    
    for t in range(T-2,-1,-1): 
        
        path[t] = back_pointeur[path[t+1]-1,t]

    return path , best_pathprob

After some tests we used the Viterbi algorithm by applying the argmax on the log of the function because, as at each iteration we multiply by $B[Obs[0],:]$ with a coordonate equals to $10e-6$ it means that after T observations we would have an $V = 10e-6n$ that converges quickly to 0 and it is impossible to do any calculus. 

In [64]:
B = np.loadtxt('PerlScriptAndModel/P.dat')

In [65]:
mails = []
paths = []
for i in range(11,31):
    mails.append(np.loadtxt('dat/mail{}.dat'.format(i), dtype=int))
    paths.append(Viterbi(mails[-1],A,B,pi)[0])
    path = paths[-1]
    for j in range(1, len(path)):
        if path[j] != path[j-1]:
            print("transition between " + str(j-1) + " and " + str(j))

transition between 2850 and 2851
transition between 2937 and 2938
transition between 2303 and 2304
transition between 4812 and 4813
transition between 2182 and 2183
transition between 1970 and 1971
transition between 2281 and 2282
transition between 2366 and 2367
transition between 2100 and 2101
transition between 1839 and 1840
transition between 2102 and 2103
transition between 2233 and 2234
transition between 2167 and 2168
transition between 2559 and 2560
transition between 2318 and 2319
transition between 2026 and 2027
transition between 1770 and 1771
transition between 2224 and 2225
transition between 2343 and 2344
transition between 2172 and 2173


In [66]:
path = Viterbi(mails[0],A,B,pi)[0]
with open('PerlScriptAndModel/path.txt','w') as f:
    for state in path:
        f.write(str(state))

In [67]:
cd ./PerlScriptAndModel/

/cal/homes/avanelst/Downloads/Data and scripts for HMM Training Session-20220608/PerlScriptAndModel


In [68]:
!perl segment.pl ../dat/mail11.txt path.txt

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [69]:
cd ..

/cal/homes/avanelst/Downloads/Data and scripts for HMM Training Session-20220608


### Q5 : How would you model the problem if you had to segment the mails in more than two parts (for example : header, body, signature) ?

We don't need to change our function nor the model but we would have to change only the arguments. We would have a HMM with 3 states (0 for the header, 1 for the body and 2 for the signature), it means that A is now a $3 \times 3$ matrix with the same idea that you can't move backward. As we would necessarily start on the header mode like the first model, we have $\pi = (1,0,0)$  and $\begin{pmatrix}
a_{1,1} & a_{1,2} & a_{1,3}\\
0 & a_{2,2} & a_{2,3}\\
0 & 0 & 1
\end{pmatrix}$ 
Also, B is now a $N \times 3$ matrix.

###  Q6 : How would you model the problem of separating the portions of mail included, knowing that they always start with the character ">".

In order to separate the portions of mail included, we could add another state called "other_mail" and the the matrix $A$ would now be a $4 \times 4$ matrix that is with 4 states : header, body, other_mail and signature. Like Q5, transitions 1->2, 2->4 and 1->4 are possible but new transitions are now possible : 2->3, 3->2, 1->3, 3->4 but the other transitions are not possible. It is indeed possible to move from body to the mail included and from the mail included to the body.

As we would necessarily start on the header mode like the first model, we have $\pi = (1,0,0,0)$  and $\begin{pmatrix}
a_{1,1} & a_{1,2} & a_{1,3} & a_{1,4}\\
0 & a_{2,2} & a_{2,3} &a_{2,4}\\
0 & a_{3,2} & a_{3,3} & a_{3,4} \\
0 & 0 & 0 & 1 
\end{pmatrix}$ 
Also, B is now a $N \times 4$ matrix.

Knowing that included mails always start with the character ">", we could use bigrams in order to recognize the combination of "\n" represented by the number 10 in ASCII and ">" represented by the number 62 in ASCII. This would require a change in our function. A combination of 10 and number different from 62 would indicate that we are no longer in the included mail. 


## BONUS : unsupervised learning


### Q7Bonus :

Here are the formulas for the Alpha-calculation :

Let $\alpha_{i}(t)=P\left(Y_{1}=y_{1}, \ldots, Y_{t}=y_{t}, X_{t}=i \mid \theta\right)$, the probability of seeing the observations $y_{1}, y_{2}, \ldots, y_{t}$ and being in state $i$ at time $t$. This is found recursively:
1. $\alpha_{i}(1)=\pi_{i} b_{i}\left(y_{1}\right)$
2. $\alpha_{i}(t+1)=b_{i}\left(y_{t+1}\right) \sum_{j=1}^{N} \alpha_{j}(t) a_{j i}$.

Since this series converges exponentially to zero, we will scale $\alpha$ by taking the log in order to avoid numerical underflowing.
So we have 
1. $\log(\alpha_{i}(1))=\log(\pi_{i}) + \log(b_{i}\left(y_{1}\right))$
2. $\log(\alpha_{i}(t+1))=\log(b_{i}\left(y_{t+1}\right))+ \log(\sum_{j=1}^{N} \exp(\log(\alpha_{j}(t)) + \log (a_{j i}))$.


In [70]:

def alpha_calculation(Obs,A,B,pi):
    N = A.shape[1] # number of states
    T = Obs.shape[0] # number of observations
    
    alphas = np.zeros((T,N))

    
    eps = np.finfo(np.double).tiny
    A_prime = np.log(A+eps)
    pi_prime = np.log(pi+eps)
    B_prime = np.log(B+eps)
 
    
    
    alphas[0,:] = pi[:] + B_prime[Obs[0],:]
    
    for t in range(1,T):
        for s in range(N):
            alphas[t,s] = B_prime[Obs[t],s] + logsumexp(A_prime[:,s] + alphas[t-1,:])
        
    return alphas



In [71]:
alphas = alpha_calculation(mails[0],A,B,pi)
alphas

array([[-5.38815493e+00, -6.86637892e+00],
       [-8.73723058e+00, -1.00077652e+01],
       [-1.17121030e+01, -1.29320717e+01],
       ...,
       [-1.32746986e+04, -1.29978429e+04],
       [-1.32786710e+04, -1.30015075e+04],
       [-1.32826434e+04, -1.30051722e+04]])

Here are the formulas for the Beta-calculation :

Let $\beta_{i}(t)=P\left(Y_{t+1}=y_{t+1}, \ldots, Y_{T}=y_{T} \mid X_{t}=i, \theta\right)$ that is the probability of the ending partial sequence $y_{t+1}, \ldots, y_{T}$ given starting state $i$ at time $t$. We calculate $\beta_{i}(t)$ as,
1. $\beta_{i}(T)=1$,
2. $\beta_{i}(t)=\sum_{j=1}^{N} \beta_{j}(t+1) a_{i j} b_{j}\left(y_{t+1}\right)$.

In [72]:
def beta_calculation(Obs,A,B):
    N = A.shape[1] # number of states
    T = Obs.shape[0] # number of observations
    
    betas = np.zeros((T,N))

    eps = np.finfo(np.double).tiny
    A_prime = np.log(A+eps)
    B_prime = np.log(B+eps)
 
    betas[T-1,:] = np.ones((1,N))
    
    for t in range(T-2,-1,-1):
        for s in range(N):
            betas[t,s] = logsumexp(A_prime[s,:] + betas[t+1,:] + B_prime[Obs[t+1],:])
        
    return betas


In [73]:
betas = beta_calculation(mails[0],A,B)
betas

array([[-1.29987841e+04, -1.36954134e+04],
       [-1.29954350e+04, -1.36922686e+04],
       [-1.29924601e+04, -1.36893415e+04],
       ...,
       [-6.94230241e+00, -6.32934995e+00],
       [-2.97134183e+00, -2.66467497e+00],
       [ 1.00000000e+00,  1.00000000e+00]])

We have the following formulas to compute $\gamma$ and $\xi$ :

$$
\gamma_{i}(t)=\frac{\alpha_{i}(t) \beta_{i}(t)}{\sum_{j=1}^{N} \alpha_{j}(t) \beta_{j}(t)}
$$

$$
\xi_{i j}(t)=\frac{\alpha_{i}(t) a_{i j} \beta_{j}(t+1) b_{j}\left(y_{t+1}\right)}{\sum_{k=1}^{N} \sum_{w=1}^{N} \alpha_{k}(t) a_{k w} \beta_{w}(t+1) b_{w}\left(y_{t+1}\right)},
$$


In [74]:
def gamma_calculation(alphas, betas):
    T,N = alphas.shape
    gammas = np.zeros((T,N))
    for t in range(T):
        den = logsumexp(alphas[t,:] + betas[t,:])
        gammas[t,:] = alphas[t,:] + betas[t,:] - den
    return np.exp(gammas)


def xi_calculation(Obs, A, B, alphas, betas):
    T,N = alphas.shape
    eps = np.finfo(np.double).tiny
    A_prime = np.log(A+eps)
    B_prime = np.log(B+eps)

    xis = np.zeros((T-1,N,N))

    for t in range(T-1):
        for i in range(N):
            for j in range(N):
                den = logsumexp(alphas[t,:] + A_prime[:,:] + betas[t+1, :] + B_prime[Obs[t+1], :])
                xis[t,i,j] = alphas[t,i] + A_prime[i,j] + betas[t+1,j] + B_prime[Obs[t+1],j] - den
    return np.exp(xis)

In [75]:
gamma = gamma_calculation(alphas, betas)
gamma

array([[1.00000000e+000, 6.54240668e-304],
       [1.00000000e+000, 6.56478863e-304],
       [1.00000000e+000, 6.58301412e-304],
       ...,
       [3.13977416e-121, 1.00000000e+000],
       [3.13524244e-121, 1.00000000e+000],
       [3.13191110e-121, 1.00000000e+000]])

In [76]:
xi = xi_calculation(mails[0], A, B, alphas, betas)
xi

array([[[1.00000000e+000, 2.24327286e-306],
        [5.07808102e-309, 6.54235590e-304]],

       [[1.00000000e+000, 1.82879987e-306],
        [6.25025868e-309, 6.56472612e-304]],

       [[1.00000000e+000, 1.74345021e-306],
        [6.57443540e-309, 6.58294838e-304]],

       ...,

       [[3.13732102e-121, 5.67105033e-124],
        [9.62506270e-309, 9.99218689e-001]],

       [[3.13279284e-121, 4.52818585e-124],
        [1.20369247e-308, 9.99218689e-001]],

       [[3.12946411e-121, 3.32873188e-124],
        [1.63568349e-308, 9.99218689e-001]]])

We have the following formulas to update $\pi$; the matrix $A$ and the matrix $B$ :

$\pi_{i}^{*}=\gamma_{i}(1)$

$$
a_{i j}^{*}=\frac{\sum_{t=1}^{T-1} \xi_{i j}(t)}{\sum_{t=1}^{T-1} \gamma_{i}(t)}
$$

$$
b_{i}^{*}\left(v_{k}\right)=\frac{\sum_{t=1}^{T} 1_{y_{t}=v_{k}} \gamma_{i}(t)}{\sum_{t=1}^{T} \gamma_{i}(t)}
$$
where
$$
1_{y_{t}=v_{k}}= \begin{cases}1 & \text { if } y_{t}=v_{k} \\ 0 & \text { otherwise }\end{cases}
$$

In [77]:
def updated_pi(gamma):
    pi = np.zeros(gamma.shape[1])
    pi = gamma[0,:]
    return pi

In [78]:
updated_pi(gamma)

array([1.00000000e+000, 6.54240668e-304])

In [79]:
def updated_A(xi,gamma):
    N = gamma.shape[1]
    A = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            num = np.sum(xi[:-1,i,j])
            denom = np.sum(gamma[:-1,i])
            A[i,j] = num/denom
    return A 


In [80]:
updated_A(xi,gamma)

array([[9.99732670e-001, 4.46518346e-004],
       [7.15656210e-279, 9.98874944e-001]])

In [81]:
def updated_B(Obs,gamma):
    N = gamma.shape[1]
    B = np.zeros((256,N))
    T = gamma.shape[0]
    for v in range(256):
        for i in range(N):
            num=0
            for t in range(T):
                if Obs[t]==v :
                    num+= gamma[t,i]
            denom = np.sum(gamma[:,i])
            B[v,i] = num/denom
    return B 

In [82]:
updated_B(mails[0],gamma)

array([[0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [1.57215692e-003, 3.21018951e-286],
       [1.79449556e-002, 2.83038502e-002],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000],
       [0.0

The results seem to be consistent (the probabilities are indeed between 0 and 1 and the updated matrix A and B are consistent with the first A and B)